### IMPORTACIONES

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re #regex para validar si existe subfijos
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy.signal import savgol_filter
from sklearn.decomposition import PCA
from numpy import trapz
from scipy.ndimage import gaussian_filter1d

### LECTURA DE ARCHIVOS

In [2]:


# Lee el archivo CSV
df = pd.read_csv('limpio.csv', delimiter=',')

# Muestra las primeras filas del DataFrame para verificar
print(df.head())


   Ramanshift  collagen  collagen.1  collagen.2  collagen.3  collagen.4  \
0     1801.26     0.117       0.123       0.098       0.097       0.115   
1     1797.41     0.118       0.124       0.099       0.098       0.116   
2     1793.55     0.119       0.124       0.100       0.098       0.117   
3     1789.69     0.118       0.122       0.099       0.097       0.117   
4     1785.84     0.118       0.121       0.099       0.096       0.116   

   collagen.5  collagen.6  collagen.7  collagen.8  ...  DNA.100  DNA.101  \
0       0.129       0.130       0.144       0.129  ...    0.154    0.150   
1       0.130       0.131       0.145       0.129  ...    0.154    0.152   
2       0.131       0.132       0.145       0.130  ...    0.155    0.153   
3       0.131       0.132       0.146       0.131  ...    0.155    0.154   
4       0.130       0.131       0.146       0.131  ...    0.155    0.155   

   DNA.102  DNA.103  DNA.104  DNA.105  DNA.106  DNA.107  DNA.108  DNA.109  
0    0.154    0.

 ### Verificamos si se tiene los subfijos al leer el archivo

In [3]:
if any(re.search(r'\.\d+$', col) for col in df.columns):
    # Si hay columnas con sufijos, eliminarlos
    df.columns = [re.sub(r'\.\d+$', '', col) for col in df.columns]
    print("Se eliminaron los sufijos numéricos de los encabezados.")
# Muestra las primeras filas del DataFrame para verificar
print(df.head())

Se eliminaron los sufijos numéricos de los encabezados.
   Ramanshift  collagen  collagen  collagen  collagen  collagen  collagen  \
0     1801.26     0.117     0.123     0.098     0.097     0.115     0.129   
1     1797.41     0.118     0.124     0.099     0.098     0.116     0.130   
2     1793.55     0.119     0.124     0.100     0.098     0.117     0.131   
3     1789.69     0.118     0.122     0.099     0.097     0.117     0.131   
4     1785.84     0.118     0.121     0.099     0.096     0.116     0.130   

   collagen  collagen  collagen  ...    DNA    DNA    DNA    DNA    DNA  \
0     0.130     0.144     0.129  ...  0.154  0.150  0.154  0.164  0.157   
1     0.131     0.145     0.129  ...  0.154  0.152  0.155  0.164  0.158   
2     0.132     0.145     0.130  ...  0.155  0.153  0.156  0.165  0.160   
3     0.132     0.146     0.131  ...  0.155  0.154  0.157  0.165  0.160   
4     0.131     0.146     0.131  ...  0.155  0.155  0.157  0.166  0.160   

     DNA    DNA    DNA    DNA 

In [4]:
unique_headers = df.columns.unique()
print("\nEncabezados únicos:")
print(unique_headers)

# Identificar los tipos únicos de valores en los encabezados
unique_types = set(col for col in df.columns if col != "Ramanshift")


Encabezados únicos:
Index(['Ramanshift', 'collagen', 'glycogen', 'lipids', 'DNA'], dtype='object')


In [ ]:
# Colores para cada tipo
colors = plt.cm.tab20.colors  # Una paleta de colores suficientemente grande
color_map = {unique: colors[i % len(colors)] for i, unique in enumerate(unique_types)}

# Graficar cada tipo una sola vez en la leyenda
plt.figure(figsize=(14, 10))

for unique_type in unique_types:
    # Filtrar las columnas correspondientes al tipo actual
    columns = [col for col in df.columns if col.startswith(unique_type)]
    
    # Graficar todas las columnas del tipo actual
    for col in columns:
        plt.plot(df['Ramanshift'], df[col], color=color_map[unique_type], alpha=0.6)
    
    # Agregar una entrada en la leyenda solo para el tipo (una vez)
    plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend

# Etiquetas y leyendas
plt.title("Espectros Raman", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad", fontsize=14)
plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
plt.grid(True)

# Mostrar la gráfica
plt.show()

### En este caso pediremos al usuario ingresar algun tipo para graficar, para tener una idea de como se ve los espectros para cada uno de los tipos existentes en el archivo.

<div class="alert alert-block alert-info">
<b>PD:</b> Aqui solo se mostraran hasta 10 como cantidad maxima de columnas para tipo, es para una referencia y no tener una carga de datos excesiva 
</div>

In [ ]:
# Configurar el tipo de espectro que se desea graficar
#tipo_espectro = input(f"Ingrese el tipo de espectro para graficar (opciones: {', '.join(unique_types)}): ").strip()
tipo_espectro = "collagen"
# Filtrar las columnas correspondientes al tipo de espectro ingresado
columnas_tipo = [col for col in df.columns if col.startswith(tipo_espectro)]

if columnas_tipo:
    # Limitar el número de columnas graficadas
    max_columns = 10
    columnas_tipo = columnas_tipo[:max_columns]

    # Reducir la cantidad de datos graficados
    sampled_df = df.iloc[::10, :]

    # Crear la gráfica
    plt.figure(figsize=(14, 8))

    # Graficar todas las líneas sin leyenda
    for col in columnas_tipo:
        plt.plot(sampled_df['Ramanshift'], sampled_df[col], alpha=0.7)

    # Añadir una entrada única en la leyenda para el tipo
    plt.plot([], [], label=tipo_espectro, color='black') 

    # Etiquetas y leyenda
    plt.title(f"Espectro Raman - {tipo_espectro} (muestra de columnas y filas)", fontsize=16)
    plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
    plt.ylabel("Intensidad", fontsize=14)
    plt.legend(title="Espectros", fontsize=10, loc='upper right', frameon=False)
    plt.grid(True)

    # Mostrar la gráfica
    plt.show()
else:
    print(f"No se encontraron columnas para el tipo de espectro '{tipo_espectro}'. Verifique el nombre e intente nuevamente.")


# <center>Analisis PCA</center>
### ¿Por qué utilizar PCA en espectros?
En datos espectroscópicos (como los Raman), los conjuntos de datos suelen tener alta dimensionalidad y las variables (picos) pueden estar correlacionadas. El PCA es útil porque:

**Reduce la dimensionalidad:** Permite analizar un número menor de variables representativas. <br>
**Captura patrones esenciales:** Identifica las características espectrales clave. <br>
**Mejora la visualización:** Ayuda a visualizar datos complejos en gráficos 2D o 3D.  <br>
**Preprocesamiento:** Facilita la clasificación o el análisis posterior (por ejemplo, identificación de muestras).

<img src=pca-analysis.gif>


### Cálculo del PCA
**Calcular la matriz de covarianza:** Representa cómo varían las variables juntas.<br>
**Obtener los valores y vectores propios:** Los valores propios determinan la importancia (varianza explicada) de cada componente, y los vectores propios indican la dirección de los nuevos ejes. <br>
**Proyección de los datos:** Transformar los datos originales en los nuevos ejes definidos por los componentes principales.

### Aplicación práctica en espectros
#### En espectros Raman:

**Objetivo:** Identificar patrones comunes entre muestras (como grupos químicos) o distinguir diferencias entre ellas. <br>
**Componentes principales:** Representan características espectrales clave que explican la mayoría de las variaciones entre los espectros. 

<div class="alert alert-block alert-danger">
<b>Limitaciones PCA:</b> Es una técnica lineal, lo que significa que no captura relaciones no lineales en los datos.
Los componentes principales pueden ser difíciles de interpretar físicamente.
Depende de la correcta estandarización y limpieza de los datos.
</div>



# <center>Fundamento matemático</center>

### 1. Matriz de datos
Dado un conjunto de datos con 
𝑚
m muestras y 
𝑛
n variables, representamos los datos en una matriz de datos 
𝑋
X de tamaño 
𝑚
×
𝑛
m×n:
$$
X = 
\begin{bmatrix}
x_{11} & x_{12} & \dots & x_{1n} \\
x_{21} & x_{22} & \dots & x_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
x_{m1} & x_{m2} & \dots & x_{mn}
\end{bmatrix}
$$

Donde cada fila es una muestra, y cada columna es una variable (por ejemplo, la intensidad de un espectro en una longitud de onda específica).

### 2. Estandarización
El PCA requiere que las variables tengan media 0 y desviación estándar 1. Para ello, estandarizamos cada variable:

$$
z_{ij} = \frac{x_{ij} - \mu_j}{\sigma_j}
$$

#### 1. Fórmula para estandarización de los datos

$$
z_{ij} = \frac{x_{ij} - \mu_j}{\sigma_j}
$$

Donde:

- \( \mu_j \): Media de la variable \( j \).
- \( \sigma_j \): Desviación estándar de la variable \( j \).

Esto nos da una nueva matriz \( Z \), estandarizada.

---



In [ ]:
# Volver a realizar la estandarización con las columnas corregidas
data_no_suffix = df.drop(columns=["Ramanshift"])  # Eliminar la columna 'Ramanshift'

# Estandarizar los datos nuevamente
scaler = StandardScaler()
data_standardized_no_suffix = scaler.fit_transform(data_no_suffix)


# Convertir la matriz estandarizada en un DataFrame para inspección
data_standardized_no_suffix_df = pd.DataFrame(data_standardized_no_suffix, columns=data_no_suffix.columns)

# Mostrar las primeras filas del DataFrame estandarizado sin sufijos
data_standardized_no_suffix_df.head()


### 3. Matriz de covarianza

La matriz de covarianza mide cómo varían las variables entre sí:

$$
C = \frac{1}{m - 1} Z^\top Z
$$

Donde:

- \( C \): Es la matriz de covarianza (\( n \times n \)).
- \( Z^\top \): La transpuesta de la matriz estandarizada \( Z \).

Cada elemento de \( C \), \( c_{ij} \), mide la covarianza entre las variables \( i \) y \( j \):

$$
c_{ij} = \frac{1}{m-1} \sum_{k=1}^m (z_{ki} - \bar{z}_i)(z_{kj} - \bar{z}_j)
$$

---



In [ ]:
# Calcular la matriz de covarianza a partir de los datos estandarizados
covariance_matrix = np.cov(data_standardized_no_suffix.T)

# Convertir la matriz de covarianza a un DataFrame para visualización
covariance_matrix_df = pd.DataFrame(
    covariance_matrix,
    index=data_no_suffix.columns,
    columns=data_no_suffix.columns
)

# Mostrar las primeras filas de la matriz de covarianza
covariance_matrix_df.head()


<div class="alert alert-block alert-info">
<b>Matriz De Covarianza Raman</b> Se ha calculado la matriz de covarianza a partir de los datos estandarizados. Esta matriz representa cómo varían las variables (columnas del espectro) entre sí.
</div>



### 4. Descomposición en valores propios

El PCA se basa en encontrar los vectores propios (\( v \)) y los valores propios (\( \lambda \)) de la matriz de covarianza:

$$
C v = \lambda v
$$

Donde:

- \( v \): Es el vector propio (dirección del nuevo eje).
- \( \lambda \): Es el valor propio (cuánta varianza explica ese eje).

Los valores propios están ordenados de mayor a menor y representan la cantidad de varianza explicada por cada componente principal.

---

In [ ]:
# Descomposición en valores propios y vectores propios
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

# Convertir los valores propios en un DataFrame para inspección
eigenvalues_df = pd.DataFrame(eigenvalues, columns=["Valor Propio"])

# Convertir los vectores propios en un DataFrame para inspección
eigenvectors_df = pd.DataFrame(
    eigenvectors,
    index=data_no_suffix.columns,
    columns=[f"PC{i+1}" for i in range(eigenvectors.shape[1])]
)

# Mostrar los valores propios y vectores propios
eigenvalues_df.head()
eigenvectors_df.head()


*La descomposición en valores propios y vectores propios de la matriz de covarianza se ha realizado correctamente:*<br>
**Valores propios (eigenvalues):** Indican la cantidad de varianza explicada por cada componente principal.<br>
**Vectores propios (eigenvectors):** Representan las direcciones (ejes) de los nuevos componentes principales en el espacio original.

### 5 Transformación de los datos

Los datos originales se transforman proyectándolos en los ejes definidos por los vectores propios:

$$
T = Z V
$$

Donde:

- \( T \): Matriz transformada (nuevos datos en el espacio de los componentes principales).
- \( V \): Matriz cuyas columnas son los vectores propios (direcciones principales).

Cada fila de \( T \) es la representación de una muestra en el espacio reducido.

---

In [ ]:
# Proyección de los datos originales en los ejes definidos por los componentes principales
transformed_data = np.dot(data_standardized_no_suffix, eigenvectors)

# Convertir los datos transformados en un DataFrame para inspección
transformed_data_df = pd.DataFrame(
    transformed_data,
    columns=[f"PC{i+1}" for i in range(transformed_data.shape[1])]
)

# Mostrar los primeros datos transformados
transformed_data_df.head()


*Resultados:* <br>
**Cada fila:** Representa una muestra en el espacio PCA. <br>
**Cada columna (PC1, PC2, ...):** Representa un componente principal.

### 6. Varianza explicada

La proporción de varianza explicada por cada componente principal es:

$$
\text{Varianza explicada} = \frac{\lambda_i}{\sum \lambda}
$$

Esto nos dice cuánto contribuye cada componente principal a la variabilidad total de los datos.

In [ ]:
# Calcular la varianza explicada por cada componente principal
explained_variance = eigenvalues / np.sum(eigenvalues)

# Calcular la varianza acumulada
cumulative_variance = np.cumsum(explained_variance)

pc1 = transformed_data[:, 0]
pc2 = transformed_data[:, 1]

# Crear un gráfico de dispersión utilizando PC1 y PC2
#plt.figure(figsize=(10, 6))
#plt.scatter(pc1, pc2, alpha=0.7, edgecolor='k')
#plt.xlabel("Componente Principal 1 (PC1)", fontsize=14)
#plt.ylabel("Componente Principal 2 (PC2)", fontsize=14)
#plt.title("Proyección de Espectros en el Espacio PCA", fontsize=16)
#plt.grid(True)

In [ ]:

# Leer el archivo CSV y excluir la primera columna (ejemplo: Ramanshift)
data = df.iloc[:, 1:]

# Obtener las categorías desde los nombres de las columnas
# Se asume que los nombres de las columnas contienen las categorías como parte del texto
categories = [col.split('_')[0] for col in data.columns]  # Divide por un delimitador como "_" (ajusta según tu formato)
unique_categories = list(set(categories))  # Categorías únicas

# Asignar un color único a cada categoría
colors = plt.cm.tab10.colors  # Paleta de colores
category_colors = {category: colors[i % len(colors)] for i, category in enumerate(unique_categories)}

# Escalar los datos
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.T)  # Transposición para que las características sean columnas

# Aplicar PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_scaled)

# Graficar el PCA con colores por tipo
plt.figure(figsize=(10, 6))
for category in unique_categories:
    indices = [i for i, cat in enumerate(categories) if cat == category]
    plt.scatter(
        pca_result[indices, 0],
        pca_result[indices, 1],
        label=category,
        color=category_colors[category],
        alpha=0.7,
        edgecolor='k'
    )

# Etiquetas y leyenda
plt.xlabel('PC1', fontsize=14)
plt.ylabel('PC2', fontsize=14)
plt.title('Proyección PCA', fontsize=16)
plt.legend(loc='best', fontsize=10)
plt.grid(True)
plt.show()


## <center>¿Por qué suavizar los datos antes del PCA?</center>

### Reducción de ruido: 

En espectros, el ruido puede distorsionar las señales reales y hacer que el PCA se enfoque en variaciones no representativas.
Suavizar elimina fluctuaciones pequeñas e irrelevantes, dejando solo las tendencias principales.<br>

### Mejor representación de los patrones:

Las características significativas (picos y valles) de un espectro se destacan mejor después del suavizado.
El PCA trabajará sobre señales más limpias y representativas, en lugar de desviaciones causadas por el ruido.

### Reducir complejidad: 

Suavizar puede disminuir la cantidad de detalles excesivos en los datos, facilitando la identificación de las principales componentes. 


### Mejor interpretación de los resultados:

Los componentes principales extraídos del PCA serán más fáciles de relacionar con características relevantes de los datos (como picos en un espectro).


## <center> ¿Cuándo no es necesario suavizar? </center> 
Si los datos ya son de alta calidad y el ruido es mínimo, el suavizado puede no ser necesario.
Un exceso de suavizado puede eliminar detalles importantes, lo que podría llevar a perder información relevante.

# <center>Métodos de suavizado comunes</center>

## **1. Savitzky-Golay Filter**
- Realiza un ajuste polinómico local en una ventana móvil.
- Ideal para espectros, ya que preserva los picos y las características de la señal.

**Fórmula:**
$$
y'(t) = \sum_{k=-m}^{m} c_k \cdot x(t+k)
$$

Donde:

- $$y'(t)$$: Señal suavizada.
- $$c_k$$: Coeficientes del polinomio.
- $$x(t+k)$$: Valores originales en la ventana móvil.

---


<img src=filtro-sg.ppm>

In [ ]:
# Parámetros para suavizar
window_length = 11  # Longitud de la ventana (debe ser impar)
polyorder = 3       # Orden del polinomio

### 1. <font color="blue">window_length:</font> Longitud de la ventana
**Definición:** Es el número de puntos consecutivos que se utilizan para ajustar un polinomio en el proceso de suavizado.
#### Requisitos:
Debe ser un número entero impar (por ejemplo, 5, 7, 9, 11, etc.).
Debe ser mayor que el orden del polinomio (polyorder).
### Impacto:
**Ventana pequeña:** El suavizado será más localizado y detallado, pero puede no eliminar bien el ruido.<br>
**Ventana grande:** El suavizado será más amplio y eliminará más ruido, pero podría borrar picos o detalles importantes.

### 2. <font color="blue">polyorder:</font> Orden del polinomio
**Definición:** Es el grado del polinomio que se ajusta a los puntos dentro de cada ventana.
### Requisitos:
Debe ser un número entero no negativo.
Debe ser menor que el tamaño de la ventana (window_length).
### Impacto:
**Orden bajo (e.g., 2 o 3):** Se ajusta a tendencias generales y no captura oscilaciones rápidas.<br>
**Orden alto (e.g., 4 o más):** Captura más detalles locales, pero puede amplificar el ruido si se usa con datos ruidosos.

In [ ]:
# Aplicar Savitzky-Golay al dataset
data_smoothed = savgol_filter(df.iloc[:, 1:], window_length=window_length, polyorder=polyorder, axis=0)

### Cómo funciona <font color="blue">savgol_filter</font>
El filtro realiza un ajuste polinómico en ventanas móviles de datos. Dentro de cada ventana, se ajusta un polinomio de un grado específico a los datos y luego se usa ese polinomio para calcular el valor suavizado del punto central.

**Ventana móvil:**<br>
Es un subconjunto de datos de longitud definida por el parámetro window_length.<br>
La ventana se mueve a través de la señal, centrada en cada punto que se va a suavizar.<br>
**Ajuste polinómico:**<br>
Dentro de cada ventana, se ajusta un polinomio de grado polyorder.<br>
El valor del punto central de la ventana se reemplaza por el valor del polinomio ajustado.<br>
**Repetición:**<br>
El proceso se repite para cada punto de la señal, excepto en los extremos, donde la ventana no puede ser completamente centrada. En esos casos, se utiliza un método de extrapolación.

In [ ]:
# Crear un DataFrame con los datos suavizados
df_smoothed = pd.DataFrame(data_smoothed, columns=df.columns[1:])
df_smoothed.insert(0, 'Ramanshift', df['Ramanshift'])  # Insertar de vuelta la columna 'Ramanshift' que quitamos para
#poder suavizar

# Obtener los tipos únicos desde los nombres de las columnas
unique_types = set(col.split('_')[0] for col in df.columns[1:])  # Ajusta el separador si es necesario

# Crear un mapa de colores
colors = plt.cm.tab20.colors  # con 20 colores si podra colorear
color_map = {unique: colors[i % len(colors)] for i, unique in enumerate(unique_types)}

# Graficar los espectros suavizados
plt.figure(figsize=(14, 10))

for unique_type in unique_types:
    # Filtrar las columnas correspondientes al tipo actual
    columns = [col for col in df.columns if col.startswith(unique_type)]
    
    # Graficar todas las columnas del tipo actual
    for col in columns:
        plt.plot(df_smoothed['Ramanshift'], df_smoothed[col], color=color_map[unique_type], alpha=0.6)
    
    # Agregar una entrada en la leyenda solo para el tipo (una vez)
    plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend

# Etiquetas y leyendas
plt.title("Espectros Raman Suavizados SG", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad Suavizada", fontsize=14)
plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
plt.grid(True)

# Mostrar la gráfica
plt.show()


## 2. Filtro Gaussiano
El filtro gaussiano es una técnica de suavizado que se utiliza para reducir el ruido en los espectros (como los espectros Raman) preservando las características principales, como los picos. Este filtro aplica una convolución entre los datos espectrales y una función gaussiana, lo que atenúa las variaciones rápidas (ruido) y retiene los patrones de baja frecuencia (picos y formas importantes).

### Cómo funciona el filtro gaussiano
**Definición de la función gaussiana:** La función gaussiana es una curva en forma de campana que da más peso a los puntos cercanos al valor central. Matemáticamente, se define como:

$$
G(x) = \frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{x^2}{2\sigma^2}}
$$

Donde: <br>
G(x): Valor de la función gaussiana en un punto <br>
σ: Desviación estándar, que determina el ancho de la curva gaussiana.

**Aplicación en el espectro:**

* Se toma una ventana alrededor de cada punto del espectro.
* Se calcula un promedio ponderado de los valores dentro de la ventana, donde los pesos están determinados por la función gaussiana. <br>

**Atenuación del ruido:**

El ruido, que tiende a tener variaciones rápidas, se atenúa debido al promedio ponderado.
Las características principales, como los picos del espectro, se preservan.


In [ ]:


# Parámetros para el filtro gaussiano
sigma = 2  # Desviación estándar del filtro

# Aplicar el filtro gaussiano a los datos espectrales
data_smoothed_gaussian = gaussian_filter1d(df.iloc[:, 1:].values, sigma=sigma, axis=0)

# Crear un DataFrame con los datos suavizados
df_smoothed_gaussian = pd.DataFrame(data_smoothed_gaussian, columns=df.columns[1:])
df_smoothed_gaussian.insert(0, 'Ramanshift', df['Ramanshift'])  # Insertar la columna 'Ramanshift'

# Obtener los tipos únicos desde los nombres de las columnas
unique_types = set(col.split('_')[0] for col in df.columns[1:])  # Ajusta el separador si es necesario

# Crear un mapa de colores
colors = plt.cm.tab20.colors  # Paleta de colores suficientemente grande
color_map = {unique: colors[i % len(colors)] for i, unique in enumerate(unique_types)}

# Graficar los espectros suavizados
plt.figure(figsize=(14, 10))

for unique_type in unique_types:
    # Filtrar las columnas correspondientes al tipo actual
    columns = [col for col in df.columns if col.startswith(unique_type)]
    
    # Graficar todas las columnas del tipo actual
    for col in columns:
        plt.plot(df_smoothed_gaussian['Ramanshift'], df_smoothed_gaussian[col], color=color_map[unique_type], alpha=0.6)
    
    # Agregar una entrada en la leyenda solo para el tipo (una vez)
    plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend

# Etiquetas y leyendas
plt.title("Espectros Raman Suavizados con Filtro Gaussiano", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad Suavizada", fontsize=14)
plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
plt.grid(True)

# Mostrar la gráfica
plt.show()


In [ ]:
# Graficar los espectros suavizados para comparar con colores diferentes por tipo
plt.figure(figsize=(20, 8))

# Crear un mapa de colores
unique_types = set(col.split('_')[0] for col in df_smoothed_gaussian.columns[1:])
colors = plt.cm.tab20.colors
color_map = {unique: colors[i % len(colors)] for i, unique in enumerate(unique_types)}

# Subplot para el filtro gaussiano
plt.subplot(1, 2, 1)
for unique_type in unique_types:
    columns = [col for col in df_smoothed_gaussian.columns if col.startswith(unique_type)]
    for col in columns:
        plt.plot(df_smoothed_gaussian['Ramanshift'], df_smoothed_gaussian[col], color=color_map[unique_type], alpha=0.6)
    plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend
plt.title("Filtro Gaussiano", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad Suavizada", fontsize=14)
plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
plt.grid(True)

# Subplot para el filtro Savitzky-Golay
plt.subplot(1, 2, 2)
for unique_type in unique_types:
    columns = [col for col in df_smoothed.columns if col.startswith(unique_type)]
    for col in columns:
        plt.plot(df_smoothed['Ramanshift'], df_smoothed[col], color=color_map[unique_type], alpha=0.6)
    plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend
plt.title("Filtro Savitzky-Golay", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad Suavizada", fontsize=14)
plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
plt.grid(True)

# Ajustar y mostrar
plt.tight_layout()
plt.show()


**Filtro de Savitzky-Golay:**
* Utiliza un ajuste polinómico en una ventana móvil.
* Si los parámetros de longitud de ventana (window_length) y orden del polinomio (polyorder) están ajustados de manera conservadora, el resultado será muy similar al de un filtro gaussiano. <br>

**Filtro Gaussiano:**<br>
* Realiza una convolución con una función gaussiana.
* El parámetro clave es 𝜎 (desviación estándar), que controla la suavidad.<bR>

<div class="alert alert-block alert-info">
Si la longitud de ventana de Savitzky-Golay y 𝜎 en el filtro gaussiano son equivalentes en términos de suavizado, las gráficas resultantes serán casi idénticas.
</div>



#### Diferencias:
Calcula la diferencia absoluta entre los datos suavizados por cada filtro:

In [ ]:
difference = df_smoothed_gaussian.iloc[:, 1:] - df_smoothed.iloc[:, 1:]
print(difference.abs().sum())

# <center>Normalizacion de los espectros</center>

Normalizar el espectro es un proceso que consiste en escalar o transformar los datos de un espectro a un rango o referencia común. Esto se hace para poder comparar diferentes espectros entre sí, y para eliminar variaciones aleatorias en la amplitud de cada intensidad.
### Métodos de Normalización y Sus Casos de Uso

| **Método**                 | **Propósito**                                                                 |
|----------------------------|------------------------------------------------------------------------------|
| **Escalado Min-Max**       | Cuando los datos necesitan estar en un rango específico (por ejemplo, [0, 1]).|
| **Z-Score (Estandarización)** | Cuando se necesita centrar y escalar los datos (por ejemplo, para PCA o agrupamiento). |
| **Normalización por Área** | Comparar formas o perfiles (por ejemplo, en espectroscopía o cromatografía).  |
| **Normalización por Máximo** | Resaltar intensidades relativas (por ejemplo, en datos espectrales).          |


In [ ]:
# 1. Normalización por valor máximo
df_normalized_max = df.iloc[:, 1:].div(df.iloc[:, 1:].max(axis=0), axis=1)
df_normalized_max.insert(0, 'Ramanshift', df['Ramanshift'])  # Reinsertar la columna 'Ramanshift'

# 2. Normalización por área bajo la curva
areas = trapz(df.iloc[:, 1:].values, x=df['Ramanshift'].values, axis=0)
df_normalized_area = df.iloc[:, 1:].div(areas, axis=1)
df_normalized_area.insert(0, 'Ramanshift', df['Ramanshift'])

# 3. Normalización Z-Score
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df.iloc[:, 1:])
df_normalized_zscore = pd.DataFrame(data_scaled, columns=df.columns[1:])
df_normalized_zscore.insert(0, 'Ramanshift', df['Ramanshift'])

# Mostrar las primeras filas de los DataFrames normalizados para inspección

df_normalized_max.head()
df_normalized_area.head()
df_normalized_zscore.head()
#print(df_normalized_zscore)

In [ ]:
df_normalized_max.to_csv('normalized_max.csv', index=False)
df_normalized_area.to_csv('normalized_area.csv', index=False)
df_normalized_zscore.to_csv('normalized_zscore.csv', index=False)

In [ ]:

# 1. Eliminar sufijos numéricos de los nombres de las columnas
df.columns = [re.sub(r'\.\d+$', '', col) for col in df.columns]

# 2. Normalización por diferentes métodos

# Normalización por valor máximo
df_normalized_max = df.iloc[:, 1:].div(df.iloc[:, 1:].max(axis=0), axis=1)
df_normalized_max.insert(0, 'Ramanshift', df['Ramanshift'])

# Normalización por área bajo la curva
areas = trapz(df.iloc[:, 1:].values, x=df['Ramanshift'].values, axis=0)
df_normalized_area = df.iloc[:, 1:].div(areas, axis=1)
df_normalized_area.insert(0, 'Ramanshift', df['Ramanshift'])

# Normalización Z-Score
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df.iloc[:, 1:])
df_normalized_zscore = pd.DataFrame(data_scaled, columns=df.columns[1:])
df_normalized_zscore.insert(0, 'Ramanshift', df['Ramanshift'])

# Función para graficar espectros
def plot_normalized_spectra(df_normalized, title):
    # Obtener tipos únicos
    unique_types = set(col.split('_')[0] for col in df_normalized.columns[1:])
    
    # Crear un mapa de colores
    colors = plt.cm.tab20.colors
    color_map = {unique: colors[i % len(colors)] for i, unique in enumerate(unique_types)}
    
    # Graficar
    plt.figure(figsize=(14, 10))
    for unique_type in unique_types:
        columns = [col for col in df_normalized.columns if col.startswith(unique_type)]
        for col in columns:
            plt.plot(df_normalized['Ramanshift'], df_normalized[col], color=color_map[unique_type], alpha=0.6)
        plt.plot([], [], label=unique_type, color=color_map[unique_type])  # Dummy plot for legend
    
    # Etiquetas y leyendas
    plt.title(title, fontsize=16)
    plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
    plt.ylabel("Intensidad Normalizada", fontsize=14)
    plt.legend(title="Tipos", fontsize=12, loc='upper right', frameon=False)
    plt.grid(True)
    plt.show()






## <center>Graficos normalizados, por tipo de normalizacion</center>

### Normalización por Máximo

La **normalización por máximo** ajusta los datos dividiendo cada valor por el valor máximo en su columna. Esto escala los valores al rango \([0, 1]\).

**Fórmula:**

$$
x_{\text{norm}} = \frac{x}{x_{\text{max}}}
$$

Donde:
- \( x \): Valor original.
- \( x_{\text{max}} \): Valor máximo de la columna correspondiente.

#### Propósito:
- Resaltar las intensidades relativas en los datos.
- Comparar espectros escalados a un rango uniforme.

In [ ]:
plot_normalized_spectra(df_normalized_max, "Espectros Normalizados por Máximo")

### Normalizados por Área
La normalización por área es un método común para escalar datos espectrales, como los espectros Raman, que ajusta cada valor del espectro dividiendo por el área total bajo la curva. Esto asegura que el área total de cada espectro sea igual a 1, permitiendo comparaciones más equitativas de las formas relativas de los espectros.

#### Cómo funciona la Normalización por Área

#### Calcular el área bajo la curva (AUC):
El área se calcula como la integral de la intensidad a lo largo de los valores del desplazamiento Raman (o eje X). 

En datos discretos, como los espectros Raman, se utiliza una suma aproximada o la **regla del trapecio** para estimar esta integral:

$$
A = \int f(x) \, dx \approx \sum_{i=1}^{n-1} \frac{f(x_{i+1}) + f(x_i)}{2} (x_{i+1} - x_i)
$$

En Python, esto se logra con la función `numpy.trapz`.

---

#### Dividir cada valor por el área total:
Una vez calculada el área (\( A \)), cada valor del espectro (\( x \)) se divide por \( A \):

$$
x_{\text{norm}} = \frac{x}{A}
$$

---

#### Resultado:
- Los valores del espectro escalados estarán en una **escala relativa**.
- El área bajo la curva del espectro normalizado será igual a **1**.


In [ ]:
plot_normalized_spectra(df_normalized_area, "Espectros Normalizados por Área")


## Normalizacion Z-Score (Estandarización)
La normalización Z-Score, también conocida como estandarización, transforma los datos para que cada variable tenga una media de 0 y una desviación estándar de 1. Este método es especialmente útil para análisis estadísticos como el PCA, donde las escalas de las variables pueden afectar el resultado.

### Fórmula del Z-Score

Cada valor se transforma usando la fórmula:

$$
z = \frac{x - \mu}{\sigma}
$$

Donde:
- \( x \): Valor original.
- \( \mu \): Media de la variable.
- \( \sigma \): Desviación estándar de la variable.


<div class="alert alert-block alert-info">
<b>Interpretación de 
𝜎
σ en espectros Raman</b> 
    <br><font color=red>Desviación estándar pequeña:</font> La intensidad de los valores está más concentrada alrededor de la media, indicando picos más homogéneos.<br>
    <font color=red>Desviación estándar grande:</font> Los valores de intensidad están más dispersos, indicando variaciones significativas en el espectro.<br>
</div>




In [ ]:
plot_normalized_spectra(df_normalized_zscore, "Espectros Normalizados por Z-Score")

# Corrección de Shirley
## ¿Por qué se necesita la corrección de Shirley?

### **Presencia de un fondo no lineal:**
- Los espectros suelen incluir un fondo causado por:
  - Efectos secundarios.
  - Ruido.
  - Respuesta del instrumento.
- Este fondo no lineal puede superponerse con los picos de interés, complicando su análisis.

### **Análisis preciso de los picos:**
- Los picos en un espectro representan características físicas o químicas clave, como:
  - Vibraciones moleculares.
  - Estados electrónicos.
- La corrección de Shirley elimina el fondo para que los picos sean más visibles y se puedan analizar con mayor precisión.

### **Estandarización:**
- Al eliminar el fondo, los espectros de diferentes muestras o instrumentos se pueden comparar directamente.

---

## ¿Cómo funciona la corrección de Shirley?
- La corrección de Shirley modela el fondo como una curva no lineal y lo ajusta iterativamente:
  1. El área **debajo de la curva ajustada** se iguala al área **sobre la curva** (en un rango definido del espectro).
  2. Se asume que la contribución del fondo **aumenta con la intensidad acumulativa** del espectro.

---

## Aplicaciones de la corrección de Shirley

### **XPS (Espectroscopía de Fotoelectrones de Rayos X):**
- Elimina el fondo causado por emisiones secundarias de electrones.
- Permite determinar con precisión la composición elemental y los estados químicos.

### **Espectroscopía Raman:**
- Corrige fondos de fluorescencia u otras señales amplias que ocultan los picos Raman.

### **Espectroscopía UV-Vis e Infrarroja:**
- Corrige líneas de base causadas por efectos de dispersión o absorción.

---

## Ventajas de la corrección de Shirley

### **Preservación de los picos:**
- Conserva la forma y el área de los picos, lo que la hace ideal para análisis cuantitativos y cualitativos.

### **Versatilidad:**
- Es efectiva para fondos no lineales, lo que la hace aplicable a una amplia variedad de técnicas espectroscópicas.

### **Ajuste iterativo:**
- Proporciona un método flexible que converge hacia una línea de base precisa.

---

## Limitaciones de la corrección de Shirley

### **Requiere tiempo:**
- Los procesos iterativos pueden ser computacionalmente costosos para conjuntos de datos grandes.

### **Dependencia de los puntos iniciales/finales:**
- Es necesario definir correctamente el rango del espectro donde se aplicará la corrección.

### **Riesgo de sobrecorrección:**
- Si no se aplica adecuadamente, puede distorsionar la señal, especialmente en espectros con picos superpuestos.


In [ ]:
def shirley_correction(raman_shift, intensity):
    """
    Aplica la corrección de Shirley para un espectro.
    
    Parameters:
        raman_shift (array-like): Valores del eje X (Raman Shift).
        intensity (array-like): Intensidades del espectro (eje Y).
        
    Returns:
        corrected_intensity (array-like): Intensidades corregidas.
    """
    if len(raman_shift) != len(intensity):
        raise ValueError("La longitud de 'Ramanshift' y la intensidad no coincide.")
    
    corrected_intensity = intensity.copy()
    start = corrected_intensity[0]
    end = corrected_intensity[-1]
    
    for _ in range(100):  # Máximo 100 iteraciones
        background = start + (end - start) * np.cumsum(corrected_intensity) / np.sum(corrected_intensity)
        corrected_intensity = intensity - background
        corrected_intensity[corrected_intensity < 0] = 0  # Evitar valores negativos
    
    return corrected_intensity

In [ ]:
raman_shift = df['Ramanshift'].values
corrected_spectra = {}

for col in df.columns[1:]:
    intensity = df[col].values
    # Validar longitud y ajustar si es necesario
    if len(raman_shift) == len(intensity):
        corrected_spectra[col] = shirley_correction(raman_shift, intensity)
    elif len(raman_shift) > len(intensity):
        # Alinear con raman_shift recortando
        intensity = np.pad(intensity, (0, len(raman_shift) - len(intensity)), constant_values=0)
        corrected_spectra[col] = shirley_correction(raman_shift, intensity)
    else:
        print(f"Advertencia: La columna '{col}' tiene más datos que 'Ramanshift'. Será recortada.")
        intensity = intensity[:len(raman_shift)]
        corrected_spectra[col] = shirley_correction(raman_shift, intensity)

# Crear un DataFrame con los datos corregidos
df_corrected = pd.DataFrame(corrected_spectra)
df_corrected.insert(0, 'Ramanshift', raman_shift)

# Graficar los espectros corregidos
plt.figure(figsize=(14, 10))
for col in df_corrected.columns[1:]:
    plt.plot(df_corrected['Ramanshift'], df_corrected[col], alpha=0.6)

plt.title("Espectros Raman Corregidos (Shirley)", fontsize=16)
plt.xlabel("Raman Shift (cm⁻¹)", fontsize=14)
plt.ylabel("Intensidad Corregida", fontsize=14)
plt.grid(True)
plt.show()